In [1]:
import subprocess
import json
import os

from web3 import Web3   #Python Ethereum library
from bit import *  #Python Bitcoin library
from bit.network import NetworkAPI
from eth_account import Account #ethereum Account
from constants import *
from dotenv import load_dotenv
from pathlib import Path

load_dotenv()

True

In [2]:
#w3 provider
w3 = Web3(Web3.HTTPProvider("http://127.0.0.1:8545"))

#Add the following middlewareto web3.py to support the PoA algorithm:
from web3.middleware import geth_poa_middleware
w3.middleware_onion.inject(geth_poa_middleware, layer=0)

#get mnemonic from .env
mnemonic = os.getenv("MNEMONIC_KEY", 'insert mnemonic here')

w3.isConnected()

False

In [3]:
print(type(mnemonic))

<class 'str'>


In [5]:
#functions

# derive wallets function that uses hd-wallet-derfie tool
def derive_wallets(mnemonic, coin, numderive):
    command = 'php ./derive -g --mnemonic="' + mnemonic + '" --numderive="' + str(numderive) + '" --cols=path,address,privkey,pubkey --coin="' + coin + '" --format=jsonpretty'
    p = subprocess.Popen(command, stdout=subprocess.PIPE, shell=True)
    output, err = p.communicate()
    p_status = p.wait()

    return json.loads(output)

# private key to accound function
def priv_key_to_account (coin, priv_key):
    if coin == ETH:
        return Account.privateKeyToAccount(priv_key)
    if coin == BTCTEST:
        return PrivateKeyTestnet(priv_key)

# create transaction function
def create_tx(coin, account, recipient, amount):
    if coin == ETH:
        gasEstimate = w3.eth.estimateGas(
            {"from": account.address, "to": recipient, "value": amount}
        )
        return {
            "from": account.address,   # DOUBLE CHECK THAT ACCOUNT.ADDRESS MAKES SENSE???
            "to": recipient,
            "value": amount,
            "gasPrice": w3.eth.gasPrice,
            "gas": gasEstimate,
            "nonce": w3.eth.getTransactionCount(account.address),
            #AND CHAIN ID???
        }

    if coin == BTCTEST:
        return PrivateKeyTestnet.prepare_transaction(account.address, [(recipient, amount, BTC)])    #should be to or receipient??
    
# send transaction function
def send_tx(coin, account, recipient, amount):
    if coin == ETH:
        raw_tx  = create_tx(coin, account, recipient, amount)
        signed_tx = account.sign_transaction(raw_tx)
        result = w3.eth.sendRawTransaction(signed_tx.rawTransaction)
        print(result.hex())
        return result.hex()
    else:
        raw_tx  = create_tx(coin,account,recipient,amount)
        signed_tx = account.sign_transaction(raw_tx)
        result = NetworkAPI.broadcast_tx_testnet(signed_tx)     
        print(result)  
        return result        


In [6]:
coins = {'eth':derive_wallets(mnemonic=mnemonic,coin=ETH,numderive=3),'btc-test': derive_wallets(mnemonic=mnemonic,coin=BTCTEST,numderive=3)}

In [7]:
print(coins)

{'eth': [{'path': "m/44'/60'/0'/0/0", 'address': '0x90A5B550676f20Ce18a90c8EBB277D82f7bd226a', 'privkey': '0xb9116b8ff6c4e538a84880fa5d8fec427551ecae8db59c3a18e76cf91e7bb75a', 'pubkey': '028b49c9e7f9263d14245adbddab1c2afb88ab237c59e039af6e9967303b4d55af'}, {'path': "m/44'/60'/0'/0/1", 'address': '0x2EbF330442eaFadB57bb7c1E970e9432B1ae8421', 'privkey': '0x69c7ddf5bccb1e536b3c3168e5ebf4d5f919a045793e45dedfd12393199090cd', 'pubkey': '0383c2e0cae4c312d780ff209b2e4d7c00041132d1fbb2ebbad946cdbb74e203b5'}, {'path': "m/44'/60'/0'/0/2", 'address': '0xfdDeD200d0A97dE9b394d58839F98596B1961D93', 'privkey': '0xd7873ea4df72bbcf742652352a9b6f993e73c6d29daa09e73847f12d639f44b5', 'pubkey': '03c92db949e80e2a7ea939c041c26d7d6c039bb2e3ecde25b77e00c2b052f2a853'}], 'btc-test': [{'path': "m/44'/1'/0'/0/0", 'address': 'mhNGadXRRE4nCUVSHDb6oywJsNEKHMfeVp', 'privkey': 'cU6U8DJqiCSQ191Chx1KnxCnLbCTmwC6joKfqZtHoZT59QxXy2mH', 'pubkey': '03fc8846e8aa50b3ee94cca225719eeb5f7249654f4d6210694500e7fce20ea4ea'}, {'path':

In [8]:
# assign private keys from above to variables
eth_privatekey = coins['eth'][0]['privkey']
btc_privatekey = coins['btc-test'][0]['privkey']

In [9]:
print(eth_privatekey)
print(btc_privatekey)

0xb9116b8ff6c4e538a84880fa5d8fec427551ecae8db59c3a18e76cf91e7bb75a
cU6U8DJqiCSQ191Chx1KnxCnLbCTmwC6joKfqZtHoZT59QxXy2mH


In [10]:
eth_account = priv_key_to_account(ETH,eth_privatekey)
btc_account = priv_key_to_account(BTCTEST,btc_privatekey)

print(eth_account)
print(btc_account)

<PrivateKeyTestnet: mhNGadXRRE4nCUVSHDb6oywJsNEKHMfeVp>


In [11]:
#2NBw6PXYPrCYfyc1EKvkSgfrJ5N3PFyh7ts

In [12]:
send_tx(BTCTEST, 'mhNGadXRRE4nCUVSHDb6oywJsNEKHMfeVp', '2NBw6PXYPrCYfyc1EKvkSgfrJ5N3PFyh7ts', .00588344)

AttributeError: 'str' object has no attribute 'address'